# 파이썬으로 파워포인트 다루기

MS 파워포인트와 워드를 다루는 파이썬 라이브러리로 가장 널리 사용되는 것은 <a href="https://python-pptx.readthedocs.io/en/latest/">python-pptx</a>와 <a href="https://python-docx.readthedocs.io/en/latest/">python-docx</a>다. 본 강의에서는 python-pptx를 활용해 파워포인트 파일을 다뤄보는 실습을 진행한다. (python-docx도 사용법이 유사하기 때문에 필요하다면 공식문서를 참고하자.)

<pre>pip install python-pptx</pre>

## 텍스트 모두 가져오기

파워포인트는 수많은 기능과 다양한 종류의 객체를 담고 있는 프로그램이기 때문에 모든 내용에 일일이 접근하는 게 쉬운 일은 아니다. 따라서 본 실습에서는 텍스트 상자(도형), 그리고 표에 있는 텍스트만 수집하는 방법을 소개한다.

프레젠테이션 안의 텍스트에 접근하려면 모든 슬라이드, 그 안의 모든 객체(텍스트 상자, 표 등), 그 안의 모든 단락 또는 표의 행/열 단위로 순차적인 탐색을 해야 하기 때문에 반복문을 중첩해서 사용한다.

하나의 파워포인트 파일에서 텍스트를 수집해 리스트에 담아보면 다음과 같다.

In [ ]:
from pptx import Presentation

텍스트 = []

prs = Presentation("files/template.pptx")

for slide in prs.slides:
    for shape in slide.shapes:
        
        # 텍스트 상자(도형)
        if shape.has_text_frame:
            for paragraph in shape.text_frame.paragraphs:
                텍스트.append(paragraph.text)
        # 표
        if shape.has_table:
            for row in shape.table.rows:
                for cell in row.cells:
                    텍스트.append(cell.text)    

print(텍스트)

만약 동일한 템플릿을 가진 여러개의 파워포인트 파일에서 추출한 텍스트를 하나의 엑셀 파일에 모아서 기록하고자 한다면, 위 코드를 모든 pptx 파일에 대해 반복 실행한 후 openpyxl을 통해 기록하면 된다. (어떤 파일에서 가져온 것인지 알기 위해 텍스트를 모으는 리스트 맨 앞에 파일 명을 추가했다.)

In [ ]:
from pptx import Presentation
from pathlib import Path
from openpyxl import Workbook

전체텍스트 = []

for 파일 in Path("files").iterdir():
    
    텍스트 = []
    텍스트.append(Path(파일).name)
    
    prs = Presentation(파일)

    for slide in prs.slides:
        for shape in slide.shapes:

            # 텍스트 상자(도형)
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    텍스트.append(paragraph.text)
            # 표
            if shape.has_table:
                for row in shape.table.rows:
                    for cell in row.cells:
                        텍스트.append(cell.text)
                        
    전체텍스트.append(텍스트)

    
wb = Workbook()
ws = wb.active

for 텍스트 in 전체텍스트:
    ws.append(텍스트)

wb.save("files/텍스트추출.xlsx")

## 텍스트 찾아 바꾸기

텍스트를 찾아 바꾸는 작업은 문자열을 다루는 <code>replace()</code>를 사용한다.

In [ ]:
텍스트 = "제 이름은 홍길동입니다."
텍스트수정 = 텍스트.replace("홍길동", "배트맨")

print(텍스트)
print(텍스트수정)

일단 하나의 파워포인트 파일에서 텍스트 찾아바꾸기를 시도해보자.

In [ ]:
from pptx import Presentation

prs = Presentation("files/template.pptx")

찾을내용 = "name"
바꿀내용 = "홍길동"

for slide in prs.slides:
    for shape in slide.shapes:
        
            # 텍스트 상자        
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        if 찾을내용 in run.text:
                            run.text = run.text.replace(찾을내용, 바꿀내용)
                                    
            # 표
            if shape.has_table:
                for row in shape.table.rows:
                    for cell in row.cells:
                        for paragraph in cell.text_frame.paragraphs:
                            for run in paragraph.runs:
                                if 찾을내용 in run.text:
                                    run.text = run.text.replace(찾을내용, 바꿀내용)
                                    
    
prs.save("files/template_수정.pptx")

여러개의 수정사항이 있을 때는 딕셔너리를 활용해보자.

In [ ]:
from pptx import Presentation

prs = Presentation("files/template.pptx")

수정사항 = {"number": "1", "name": "배트맨", "date": "2021년 6월 4일"}

for 찾을내용, 바꿀내용 in 수정사항.items():
    
    for slide in prs.slides:
        for shape in slide.shapes:

            # 텍스트 상자        
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        if 찾을내용 in run.text:
                            run.text = run.text.replace(찾을내용, 바꿀내용)

            # 표
            if shape.has_table:
                for row in shape.table.rows:
                    for cell in row.cells:
                        for paragraph in cell.text_frame.paragraphs:
                            for run in paragraph.runs:
                                if 찾을내용 in run.text:
                                    run.text = run.text.replace(찾을내용, 바꿀내용)

    
prs.save("files/template_수정.pptx")

# 실습: 템플릿을 활용해 개별 파워포인트 파일 생성하기

엑셀 명단에서 준비된 값을 읽어와 개인별 파워포인트 파일을 만들어보자. (이 때 엑셀에 포함된 데이터가 문자열이 아니라면 <code>str()</code>함수를 써서 문자열로 바꿔줄 필요가 있다.)

In [ ]:
from openpyxl import load_workbook
from pptx import Presentation

wb = load_workbook("files/명단.xlsx", data_only=True)
ws = wb.active

전체데이터 = []

for 행 in ws.iter_rows(min_row=2):
    
    행데이터 = {}
    
    행데이터["number"] = str(행[0].value)
    행데이터["name"] = 행[1].value
    행데이터["date"] = 행[2].value

    전체데이터.append(행데이터)

    

for 행데이터 in 전체데이터:
    
    prs = Presentation("files/template.pptx")

    for 찾을내용, 바꿀내용 in 행데이터.items():

        for slide in prs.slides:
            for shape in slide.shapes:

                # 텍스트 상자        
                if shape.has_text_frame:
                    for paragraph in shape.text_frame.paragraphs:
                        for run in paragraph.runs:
                            if 찾을내용 in run.text:
                                run.text = run.text.replace(찾을내용, 바꿀내용)
                # 표
                if shape.has_table:
                    for row in shape.table.rows:
                        for cell in row.cells:
                            for paragraph in cell.text_frame.paragraphs:
                                for run in paragraph.runs:
                                    if 찾을내용 in run.text:
                                        run.text = run.text.replace(찾을내용, 바꿀내용)
    
    prs.save(f'files/template_{행데이터["number"]}_{행데이터["name"]}.pptx')